In [27]:
import os
import string

import re

from tika import parser

import spacy
# from spacy.lang.en import English
# nlp = English()#
nlp = spacy.load('en_core_web_sm')
from spacy.lang.en.stop_words import STOP_WORDS


import pandas as pd
import numpy as np


In [2]:
!pwd

/Users/ryanmcgranaghan/Documents/Helio_ECIP/dev/Helio-KNOW/ADS_enrichment


In [3]:
base_filepath = '../data/'
file_decadal = os.path.join(base_filepath,'Heliophysics Decadal Survey -- Solar and Space Physics A Science for a Technological Society.pdf')

parsed_decadal = parser.from_file(file_decadal)


2022-07-12 14:00:47,708 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /var/folders/fr/9svwksbd6h115gvfd3t2xrd00000gq/T/tika-server.jar.
2022-07-12 14:00:52,992 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /var/folders/fr/9svwksbd6h115gvfd3t2xrd00000gq/T/tika-server.jar.md5.
2022-07-12 14:00:53,273 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [4]:
content_decadal = parsed_decadal['content']

In [19]:
len('WSO Wilcox Solar Observatory')

28

In [16]:
# idx_acronyms_start = content_decadal.index('Acronyms')
idx_acronyms_start = content_decadal.index('AAS American Astronomical Society')
idx_acronyms_end = content_decadal.index('WSO Wilcox Solar Observatory')
print('start acronyms index = {}; end acronyms index = {}'.format(idx_acronyms_start,idx_acronyms_end))

start acronyms index = 1360526; end acronyms index = 1373643


In [22]:
acronym_text_data = content_decadal[idx_acronyms_start:idx_acronyms_end+28].split('\n')
acronym_text_data


['AAS American Astronomical Society',
 'ACE Advanced Composition Explorer',
 'ACR anomalous cosmic ray',
 'AFOSR Air Force Office of Scientific Research',
 'AFRL Air Force Research Laboratory',
 'AFWA Air Force Weather Agency',
 'AGS Atmospheric and Geospace Sciences Division',
 'AGU American Geophysical Union',
 'AIA Atmospheric Imaging Assembly',
 'AIM atmosphere-ionosphere-magnetosphere (system); Aeronomy of Ice in the Mesosphere ',
 '',
 '(mission)',
 'AIMI atmosphere-ionosphere-magnetosphere interactions',
 'AIP American Institute of Physics',
 'AMISR Advanced Modular Incoherent Scatter Radar',
 'AMPERE Active Magnetosphere and Planetary Electrodynamics Response Experiment',
 'AMPTE Active Magnetospheric Particle Tracer Explorers',
 'AO Announcement of Opportunity',
 'APS American Physical Society; active pixel sensor',
 'AST Division of Astronomical Sciences',
 'ATST Advanced Technology Solar Telescope',
 'AU astronomical unit',
 'AURA Association of Universities for Research in 

In [30]:
acronym_list = []
term_list = []

for l in range(len(acronym_text_data)):
    if not acronym_text_data[l]:
        print('skipping line: {}'.format(acronym_text_data[l]))
        continue
    elif ( ('http' in acronym_text_data[l]) or (': A' in acronym_text_data[l]) or ('Copyright' in acronym_text_data[l]) or ('APPENDIX G' in acronym_text_data[l]) ):
        print('skipping line: {}'.format(acronym_text_data[l]))
        continue
    else:
        try:
            acronym_list.append(acronym_text_data[l].split(" ",1)[0])
            term_list.append(acronym_text_data[l].split(" ",1)[1])
        except:
            print('Error with line {}'.format(acronym_text_data[l]))
            acronym_list.pop(-1)
            
            
#Create the dataframe
df_acronyms = pd.DataFrame(columns=['acronyms','terms'])
df_acronyms['acronyms'] = acronym_list
df_acronyms['terms'] = term_list


skipping line: 
Error with line (mission)
skipping line: 
skipping line: 
skipping line: http://www.nap.edu/13060
skipping line: 
skipping line: 
skipping line: Solar and Space Physics: A Science for a Technological Society
skipping line: 
skipping line: Copyright National Academy of Sciences. All rights reserved.
skipping line: 
skipping line: 402 SOLAR AND SPACE PHYSICS: A SCIENCE FOR A TECHNOLOGICAL SOCIETY
skipping line: 
skipping line: 
skipping line: 
skipping line: 
skipping line: http://www.nap.edu/13060
skipping line: 
skipping line: 
skipping line: Solar and Space Physics: A Science for a Technological Society
skipping line: 
skipping line: Copyright National Academy of Sciences. All rights reserved.
skipping line: 
skipping line: APPENDIX G 403
skipping line: 
skipping line: 
skipping line: 
skipping line: 
skipping line: 
skipping line: http://www.nap.edu/13060
skipping line: 
skipping line: 
skipping line: Solar and Space Physics: A Science for a Technological Society
skip

In [33]:
df_acronyms

,acronyms,terms
0,AAS,American Astronomical Society
1,ACE,Advanced Composition Explorer
2,ACR,anomalous cosmic ray
3,AFOSR,Air Force Office of Scientific Research
4,AFRL,Air Force Research Laboratory
...,...,...
295,WHI,Whole Heliosphere Interval
296,WLC,White Light Coronagraph
297,WPI,wave-particle interaction
298,WSA-Enlil,Wang-Sheely-Arge-Enlil


In [36]:
df_acronyms.to_csv('data/solar_physics_acronyms.csv',index=False)